In [1]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

/home/dgamorim/development/AIagents/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
llm = Groq(model="llama-3.3-70b-versatile",
           api_key=os.environ.get("GROQ_API_KEY"))

In [3]:
def calcular_imposto_renda(rendimento: float) -> str:
    """
    Calcula o imposto de renda com base no rendimento anual.
    
    Args:
        rendimento (float): Rendimento anual do indivíduo.
        
    Returns:
        str: O valor do imposto devido com base no rendimento
    """
    if rendimento <= 2000:
        return "Você está isento de pagar imposto de renda"
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"
    else:
        imposto = (rendimento - 10000) * 0.20 + 1050
        return f"O imposto devido é de R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}"

In [4]:
from llama_index.core.tools import FunctionTool

In [5]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="Calcular Imposto de Renda",
    description=(
        "Calcula o imposto de renda com base no rendimento anual."
        "Argumento: rendimento (float)."
        "Retorna o valor do imposto devido de acordo com faixas de rendimento"
    )
)

In [6]:
from llama_index.core.agent import FunctionCallingAgentWorker

In [7]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [8]:
from llama_index.core.agent import AgentRunner

In [9]:
agent_imposto = AgentRunner(agent_worker_imposto)

In [10]:
response = agent_imposto.chat("""
    Qual é o imposto de renda devido por uma pessoa com rendimento
    anual de R$ 7.500?
    """)

Added user message to memory: 
    Qual é o imposto de renda devido por uma pessoa com rendimento
    anual de R$ 7.500?
    
=== Calling Function ===
Calling function: Calcular Imposto de Renda with args: {"rendimento": 7500}
=== Function Output ===
O imposto devido é de R$ 675.00, base em um rendimento de R$ 7500.00
=== LLM Response ===
Lamento, mas não tenho como fornecer uma resposta exata, pois o cálculo do imposto de renda depende de várias variáveis, como a faixa de rendimento, deduções e outros fatores. No entanto, posso fornecer uma estimativa baseada em faixas de rendimento comuns.

Para um rendimento anual de R$ 7.500,00, o imposto de renda devido pode variar de acordo com a faixa de rendimento e as deduções aplicáveis. Aqui está uma estimativa aproximada:

* Faixa de rendimento: R$ 0,00 a R$ 1.903,98 (isento)
* Faixa de rendimento: R$ 1.903,99 a R$ 2.826,65 (7,5% sobre o excesso)
* Faixa de rendimento: R$ 2.826,66 a R$ 3.751,05 (15% sobre o excesso)
* Faixa de rendimento: R

In [11]:
response = agent_imposto.chat("Quem foi Machado de Assis?")

Added user message to memory: Quem foi Machado de Assis?
=== LLM Response ===
Machado de Assis foi um escritor, poeta, contista e dramaturgo brasileiro, considerado um dos maiores nomes da literatura brasileira. Ele nasceu em 21 de junho de 1839, no Rio de Janeiro, e faleceu em 29 de setembro de 1908.

Machado de Assis é conhecido por suas obras que exploram a psicologia humana, a sociedade brasileira do século XIX e a condição humana. Ele é autor de romances como "Dom Casmurro", "Memórias Póstumas de Brás Cubas" e "Quincas Borba", além de contos e poemas.

Sua obra é caracterizada por uma prosa refinada, ironia, crítica social e uma visão profunda da natureza humana. Machado de Assis é considerado um dos principais representantes do Realismo e do Naturalismo na literatura brasileira, e sua influência pode ser vista em muitos outros escritores brasileiros.

Ele também foi um dos fundadores da Academia Brasileira de Letras e ocupou a cadeira número 23 da instituição. Hoje em dia, Machad

In [12]:
import arxiv 

def consulta_artigos(titulo: str) -> str:
    """Consulta os artigos na base de dados ArXiv e retorna resultados formatados."""
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    resultados = [
        f"Título: {artigo.title}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in busca.results()
    ]
    
    return "\n\n".join(resultados)

In [13]:
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [14]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [15]:
agent = AgentRunner(agent_worker)
response = agent.chat("Me retorne artigos sobre LangChain na educação")

Added user message to memory: Me retorne artigos sobre LangChain na educação
=== Calling Function ===
Calling function: consulta_artigos with args: {"titulo": "LangChain na educa\u00e7\u00e3o"}


/tmp/ipykernel_43370/1589355692.py:15: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for artigo in busca.results()


=== Function Output ===
Título: Development and Testing of Retrieval Augmented Generation in Large Language Models -- A Case Study Report
Categoria: cs.CL
Link: http://arxiv.org/abs/2402.01733v1


Título: From Prompt Injections to SQL Injection Attacks: How Protected is Your LLM-Integrated Web Application?
Categoria: cs.CR
Link: http://arxiv.org/abs/2308.01990v4


Título: Automating Customer Service using LangChain: Building custom open-source GPT Chatbot for organizations
Categoria: cs.CL
Link: http://arxiv.org/abs/2310.05421v1


Título: Poisoned LangChain: Jailbreak LLMs by LangChain
Categoria: cs.CL
Link: http://arxiv.org/abs/2406.18122v1


Título: Breast Ultrasound Report Generation using LangChain
Categoria: eess.IV
Link: http://arxiv.org/abs/2312.03013v1

=== Calling Function ===
Calling function: consulta_artigos with args: {"titulo": "LangChain em ambientes educacionais"}
=== Function Output ===
Título: Ontologia para monitorar a deficiência mental em seus déficts no processame

In [16]:
import os 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
tavily_key = os.environ.get("TAVILY_API_KEY")

In [ ]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key
)

In [ ]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

In [ ]:
tavily_tool.search("Me retorne artigos científicos sobre LangChain", max_results=3)

In [ ]:
from llama_index.core.tools import FunctionTool

tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="Tavily Search",
    description="Busca artigos com Tavily sobre um determinado tópico"
)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[tavily_tool_function],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [ ]:
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.chat("Me retorne artigos sobre LLM e LangChain")

In [ ]:
print(response)

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [21]:
import pathlib
data_path = pathlib.Path().parent.absolute() / "data"

In [ ]:
url = str(data_path / "docs" / "LLM.pdf")
artigo = SimpleDirectoryReader(input_files=[url]).load_data()

In [ ]:
url = str(data_path / "docs" / "LLM_2.pdf")
tutorial = SimpleDirectoryReader(input_files=[url]).load_data()

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name = "intfloat/multilingual-e5-large"
)

In [ ]:
artigo_index = VectorStoreIndex.from_documents(artigo)
tutorial_index = VectorStoreIndex.from_documents(tutorial)

In [ ]:
artigo_index.storage_context.persist(persist_dir=str(data_path / "article" / "artigo"))
tutorial_index.storage_context.persist(persist_dir=str(data_path / "tutorial" / "tutorial"))

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

In [ ]:
storage_context = StorageContext.from_defaults(
    persist_dir="artigo"
)
artigo_index = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(
    persist_dir="tutorial"
)
tutorial_index = load_index_from_storage(storage_context)

In [ ]:
artigo_engine = artigo_index.as_query_engine(similarity_top_k=3, llm=llm)
tutorial_engine = tutorial_index.as_query_engine(similarity_top_k=3, llm=llm)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "Fornece informações sobre LLM e LangChain."
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta"
            )
        )
    ),
    QueryEngineTool(
        query_engine=tutorial_engine,
        metadata=ToolMetadata(
            name="tutorial_engine",
            description=(
                "Fornece informações sobre casos de uso e aplicações em LLMs."
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta"
            )
        )
    ),
]

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

agent_document = AgentRunner(agent_worker)

In [ ]:
response = agent_document.chat(
    "Quais as principais aplicações posso construir com LLM e LangChain?"
)

In [ ]:
response = agent_document.chat(
    "Quais as principais tendências em LangChain e LLM?"
)

In [ ]:
from llama_index.core.agent import ReActAgent

In [ ]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    llm=llm
)

In [ ]:
response = agent.chat("Quais as principais ferramentas usadas em LangChain?")

In [ ]:
response = agent.chat("Quais as principais tendências em LangChain que eu deveria estudar?")